# Projeto 1 - Ciência dos Dados

Nome: Fernando Bichuette Assumpção

Nome: Felipe Lacombe 

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [20]:
%matplotlib inline
import matplotlib.pyplot as plt
import tweepy
import math
import os.path
import pandas as pd
import json
import re
import string
import emoji_list
import functools
import operator
from time import sleep
from random import shuffle

In [21]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Fernando\Documents\Insper\3° Semestre\Ciencia dos Dados\C_Dados_Projeto1


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [22]:
#importando database
dados=pd.read_excel("bolsa.xlsx",'Teste')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

No projeto 1 de Ciência de dados será feito um clasificador que vai analizar a palavra bolsa em todos os tweets e classificar se é relevante quando está relacionada a **bolsa de valores** e irrelevante quando é empregada em outro em outro sentido ex: bolsa familia, bolsa utilitaria e etc.


## Limpando database

In [23]:
#fazendo a limpeza do arquivo
titulo='Teste'

def cleanup(tabela):
    
    tabela= tabela.replace("ç","c").replace("â","a").replace("ã","a").replace("à","a")\
    .replace("ê","e").replace('ú','u').replace("é","e").replace("ô","o").replace("õ","o").replace("/"," ")\
    .replace("?"," ").replace("!"," ").replace("@"," ").replace("\n"," ").replace(";"," ")\
    .replace(",","").replace(":"," ").replace("["," ").replace("]"," ").replace("}"," ")\
    .replace("{"," ").replace("\\"," ").replace(' " '," ").replace(" ' ","").replace(" * "," ")\
    .replace(" _ "," ").replace(" - "," ").replace(" . "," ").replace(" rt "," ").replace("…","")\
    .replace("“","").replace("             "," ").replace("   ","").replace(".."," ")\
    .replace("..."," ").replace("...."," ").replace("....."," ").replace("     "," ")\
    .replace("\ ","").replace("*","").replace(")","").replace(' " '," ").replace(" ' "," ")\
    .replace(' "'," ").replace('" '," ").replace('_',' ')\
    .replace('"'," ").replace('ó','o').replace('á','a')
        
    return tabela 

dados['Teste']=dados['Teste'].apply(cleanup)



def separando_emoji(tabela,titulo):
    i = tabela[titulo]
    i = i.str.lower()
    emojis = emoji_list.all_emoji
    for emoji in emojis:
        try:
            i = i.str.replace(emoji," {} ".format(emoji) )
        except:
            pass
    tabela[titulo] = i
    return tabela

dados_emoji=separando_emoji(dados,titulo)

def separador(tabela,titulo):
    i = tabela[titulo].str.split(' ')

    nova = pd.DataFrame()
    nova['Tabela splitted'] = []

    for lista in i:
        for e in ['#','' , ' ', 'rt']:
            while lista.count(e) != 0:
                lista.remove(e)

        for palavra in lista:
            if palavra[:1] == "@":
                lista.remove(palavra)
            if palavra[:4] == 'http':
                lista.remove(palavra)
        nova = nova.append({'Tabela splitted': lista}, ignore_index=True)
    return nova


def tabela_palavras(tabela):
    a = tabela['Tabela splitted']
    prob = pd.DataFrame()
    prob['Tabela splitted'] = []
    for lista in a:
        for e in lista:
            prob = prob.append({'Tabela splitted': e}, ignore_index=True)
    
    return prob['Tabela splitted']

dados_limpos=separador(dados_emoji,titulo)

dados_limpos.head()


,Tabela splitted
0,"[agora, sigo, sem, bolsa, nenhuma, pq, as, dua..."
1,"[entrei, na, mr, cat, hoje, pra, comprar, uma,..."
2,"[zedassilva, zedassilva, e, um, disparate, sem..."
3,"[nos, ultimos, 1, 2, mesesibov, vs, cdi, cdi, ..."
4,"[luizabatista, bolsa, de, agua, quente, ou, ba..."


___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [24]:
#Contando quantas vezes cada palavra aparece
dados_formatados=tabela_palavras(dados_limpos)
palavras=dados_formatados.value_counts()
palavras.head()

bolsa    198
e        170
de       163
a        137
que       94
Name: Tabela splitted, dtype: int64

In [25]:
#Dataframe relevante 
relevantes = dados.loc[(dados['B1'])==1]
relevantes.set_index('B1',inplace=True)
relevantes.head()

,Teste
B1,
1,nos ultimos 1 2 mesesibov vs cdi cdi ibov ...
1,apos tombo de segunda bolsa tem recuperacao e ...
1,entenda por que a bolsa de valores cai e o dol...
1,bolsa de sao paulo suspende negocios tras desp...
1,tereza parou a aula pra falar do coronga e da ...


In [26]:
#Dataframe irrelevante
irrelevantes = dados.loc[(dados['B1'])==0]
irrelevantes.set_index('B1',inplace=True)
irrelevantes.head()

,Teste
B1,
0,agora sigo sem bolsa nenhuma pq as duas que eu...
0,entrei na mr cat hoje pra comprar uma bolsa de...
0,zedassilva zedassilva e um disparate sem so...
0,luizabatista bolsa de agua quente ou banho b...
0,os crente gritando na minha cara aq na igreja ...


In [27]:
#Split os tweets dos dataframes criados acima
relevante_separado=separador(relevantes,titulo)
irrelevante_separado=separador(irrelevantes,titulo)

relevante_separado.head()

,Tabela splitted
0,"[nos, ultimos, 1, 2, mesesibov, vs, cdi, cdi, ..."
1,"[apos, tombo, de, segunda, bolsa, tem, recuper..."
2,"[entenda, por, que, a, bolsa, de, valores, cai..."
3,"[bolsa, de, sao, paulo, suspende, negocios, tr..."
4,"[tereza, parou, a, aula, pra, falar, do, coron..."


In [28]:
#calculando quantas vezes cada palavra aparece nos tweets relevantes
splited_relevante=tabela_palavras(relevante_separado)
prob_relevante = splited_relevante.value_counts(True) 
num_palavras_relevante = splited_relevante.value_counts()
num_palavras_relevante.head()

bolsa    87
de       81
e        80
a        79
o        44
Name: Tabela splitted, dtype: int64

In [29]:
#calculando quantas vezes cada palavra aparece nos tweets irrelevantes
splited_irrelevante=tabela_palavras(irrelevante_separado)
prob_irrelevante = splited_irrelevante.value_counts(True) 
num_palavras_irrelevante = splited_irrelevante.value_counts()
num_palavras_irrelevante.head()

bolsa    111
e         90
de        82
a         58
que       51
Name: Tabela splitted, dtype: int64

In [30]:
# probabilidade de ocorrencia da palavra bolsa dentro do database relevante
prob_relevante['bolsa']

0.037275064267352186

In [31]:
#calculando a quantidade total de palavras temos em nosso database
tamanho=[]
novo_dados=dados_limpos.iloc[0:,0]
for t in novo_dados:
    for b in t:
        tamanho.append(b)

In [32]:
#criando novas colunas para o database final
dados_limpos["Classificacao Manual"] = pd.Series(dados["B1"])
dados_limpos["Classificacao_Naive"] = pd.Series()
dados_limpos["Resposta"] = pd.Series()

### Suavização de Laplace

In [33]:
#criando um contador para percorrer o database linha por linha
a=0
#criando as variaveis que serão usadas na suavização de Laplace
total=len(tamanho)
total_relevantes=len(num_palavras_relevante)
total_irrelevantes=len(num_palavras_irrelevante)
len_01=len(num_palavras_relevante)
len_00=len(num_palavras_irrelevante)

#fazendo a suavização de Laplace
for valor in dados_limpos["Tabela splitted"]:
    alpha = 1
    prob01 = 1
    prob00 = 1
    for val in valor:
        #caso a palavra não exista no nosso database, terá seu valor alterado, para uma melhor 
        #classificação do Naive Bayers
        if val not in num_palavras_relevante or val not in num_palavras_irrelevante:
            sec01 = alpha
            sec00 = alpha
        
        else:
            sec00=num_palavras_irrelevante[val] + alpha
            sec01=num_palavras_relevante[val] + alpha
            
            
        if val in num_palavras_relevante:
            prob01 *= (num_palavras_relevante[val] + alpha) / (total_relevantes + total*alpha)
        else: 
            prob01 *= sec01/(len(num_palavras_relevante)+total*alpha)
        
        if val in num_palavras_irrelevante:   
            prob00 *= (num_palavras_irrelevante[val] + alpha) / (total_irrelevantes + total*alpha)
        else:
            prob00 *= sec00/(len(num_palavras_irrelevante)+total*alpha)
       
    if prob01>prob00:
        prob_x = 1
        dados_limpos["Classificacao_Naive"][a] = prob_x
        if dados_limpos["Classificacao Manual"][a] == prob_x:
            dados_limpos["Resposta"][a] = "Positivo Verdadeiro "
        elif dados_limpos["Classificacao Manual"][a] != prob_x:
            dados_limpos["Resposta"][a] = "Falso Positivo"
            
    elif prob01<prob00:
        prob_x = 0
        dados_limpos["Classificacao_Naive"][a] = prob_x
        if dados_limpos["Classificacao Manual"][a] == prob_x:
            dados_limpos["Resposta"][a] ="Negativo Verdadeiro"
        elif dados_limpos["Classificacao Manual"][a] != prob_x:
            dados_limpos["Resposta"][a] ="Falso Negativo"
    a+=1
resultado=dados_limpos

C:\Users\Fernando\Anaconda3\lib\site-packages\ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Fernando\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Fernando\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Fernando\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [34]:
resultado.head()

,Tabela splitted,Classificacao Manual,Classificacao_Naive,Resposta
0,"[agora, sigo, sem, bolsa, nenhuma, pq, as, dua...",0,0.0,Negativo Verdadeiro
1,"[entrei, na, mr, cat, hoje, pra, comprar, uma,...",0,0.0,Negativo Verdadeiro
2,"[zedassilva, zedassilva, e, um, disparate, sem...",0,0.0,Negativo Verdadeiro
3,"[nos, ultimos, 1, 2, mesesibov, vs, cdi, cdi, ...",1,1.0,Positivo Verdadeiro
4,"[luizabatista, bolsa, de, agua, quente, ou, ba...",0,0.0,Negativo Verdadeiro


In [35]:
#checando a diferença entre as classificações feitas à mão e as definidas pelo Naive Byers
validador = pd.crosstab(resultado['Classificacao Manual'], resultado['Classificacao_Naive'])
validador.head()

Classificacao_Naive,0.0,1.0
Classificacao Manual,,
0,111,2
1,1,86


In [36]:
#Classificação feita na mão
dados.B1.value_counts()

0    113
1     87
Name: B1, dtype: int64

In [37]:
resposta = resultado["Resposta"].value_counts(True)
print(resposta*100)

Negativo Verdadeiro     55.5
Positivo Verdadeiro     43.0
Falso Positivo           1.0
Falso Negativo           0.5
Name: Resposta, dtype: float64


___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**